In [1]:
from STvAR import *
import STvARFileWrite as stfw

In [2]:
DiffOrder = 4  
nghostcells = 4  
KOsigma = 0.1   

In [3]:
dim = 3
path = "../Source/InitialDataSolver/"

# RHS

In [4]:
fileRHS = open(path+"ET_Integration_Rhs_K.H", "w+")

In [5]:
fileRHS.write(stfw.RHS_Header())

743

In [6]:
stVar.declState = [] #We will hold a list of evolution variable names that we cycle through when converting sympy expressions to C code.
u = stVar('u', state = True)

In [7]:
fileRHS.write(u.AMReXSymb2State())

53

In [8]:
fileVARS = open(path+"ET_Integration_Variables.H","w+")
fileVARS.write(stfw.VarString(stVar.declState))
fileVARS.close()

In [9]:
fileRHS.write(DstVar(u, difftype = 1, orderD = DiffOrder).AMReXSymb2Expr())
fileRHS.write(DstVar(u, difftype = 2, orderD = DiffOrder).AMReXSymb2Expr())
fileRHS.write(DstVar(u, 'KO', orderD = DiffOrder).AMReXSymb2Expr())

836

In [10]:
x = stVarRank1('x')  #Defining symbols for the grid point positions.  They are already declared in the header.
r = stVar('r')  #Define symbol for radius from the center of our coordinate system.
r.expr = sp.sqrt(x.symb[0]**2+x.symb[1]**2+x.symb[2]**2)  #Defining how r is related to x0, x1, and x2.

#We place two black holes at postions along the x0 = x axis.
c1 = stVarRank1('c1')
c2 = stVarRank1('c2')
c1.expr = np.array([1.168642873,0,0])
c2.expr = np.array([-1.168642873,0,0])

#Bare masses
M1 = stVar('M1')
M2 = stVar('M2')

M1.expr = 0.453*4/3
M2.expr = 0.453*2/3

#Momenta
P1 = stVarRank1('P1')
P2 = stVarRank1('P2')

P1.expr = np.array([0,0.3231917498,0])
P2.expr = np.array([0,-0.3231917498,0])

S1 = stVarRank1('S1')
S2 = stVarRank1('S2')

S1.expr = [0,0,0.2]
S2.expr = [0,0.3,0]

#To simplify expressions below we can specify intermediate coordinates defined relative to the black hole centers.
xc1 = stVarRank1('xc1')
xc2 = stVarRank1('xc2')

for i in range(3):
    xc1.expr[i] = x.symb[i]-c1.symb[i]
    xc2.expr[i] = x.symb[i]-c2.symb[i]

rc1 = stVar('rc1')
rc1.expr = sp.sqrt(xc1.symb[0]**2 + xc1.symb[1]**2 + xc1.symb[2]**2)

rc2 = stVar('rc2')
rc2.expr = sp.sqrt(xc2.symb[0]**2 + xc2.symb[1]**2 + xc2.symb[2]**2)

n1 = stVarRank1('n1')
n2 = stVarRank1('n2')

for i in range(3):
    n1.expr[i] = xc1.symb[i]/rc1.symb
    n2.expr[i] = xc2.symb[i]/rc2.symb

In [11]:
fileRHS.write(r.AMReXSymb2Expr())
fileRHS.write(c1.AMReXSymb2Expr())
fileRHS.write(c2.AMReXSymb2Expr())
fileRHS.write(M1.AMReXSymb2Expr())
fileRHS.write(M2.AMReXSymb2Expr())
fileRHS.write(P1.AMReXSymb2Expr())
fileRHS.write(P2.AMReXSymb2Expr())
fileRHS.write(S1.AMReXSymb2Expr())
fileRHS.write(S2.AMReXSymb2Expr())
fileRHS.write(xc1.AMReXSymb2Expr())
fileRHS.write(xc2.AMReXSymb2Expr())
fileRHS.write(rc1.AMReXSymb2Expr())
fileRHS.write(rc2.AMReXSymb2Expr())
fileRHS.write(n1.AMReXSymb2Expr())
fileRHS.write(n2.AMReXSymb2Expr())

108

In [12]:
Abar1UU = stVarRank2('Abar1UU')
Abar2UU = stVarRank2('Abar2UU')
for i in range(3):
    for j in range(3):
        Abar1UU.expr[i][j] += 3/(2*(rc1.symb)**2)*(P1.symb[i]*n1.symb[j]+P1.symb[j]*n1.symb[i])
        Abar2UU.expr[i][j] += 3/(2*(rc2.symb)**2)*(P2.symb[i]*n2.symb[j]+P2.symb[j]*n2.symb[i])
        for k in range(3):
            Abar1UU.expr[i][j] += -3/(2*(rc1.symb)**2)*(sp.eye(3)[i,j]-n1.symb[i]*n1.symb[j])*n1.symb[k]*P1.symb[k]
            Abar2UU.expr[i][j] += -3/(2*(rc2.symb)**2)*(sp.eye(3)[i,j]-n2.symb[i]*n2.symb[j])*n2.symb[k]*P2.symb[k]
            
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                Abar1UU.expr[i][j] +=3/(rc1.symb**3)*(n1.symb[i]*sp.Eijk(j,k,l)*S1.symb[k]*n1.symb[l]+n1.symb[j]*sp.Eijk(i,k,l)*S1.symb[k]*n1.symb[l])
                Abar2UU.expr[i][j] +=3/(rc2.symb**3)*(n2.symb[i]*sp.Eijk(j,k,l)*S2.symb[k]*n2.symb[l]+n2.symb[j]*sp.Eijk(i,k,l)*S2.symb[k]*n2.symb[l])

In [13]:
AbarUU = stVarRank2('AbarUU')
for i in range(3):
    for j in range(3):
        AbarUU.expr[i][j] += Abar1UU.symb[i][j]+Abar2UU.symb[i][j]

In [14]:
fileRHS.write(Abar1UU.AMReXSymb2Expr())
fileRHS.write(Abar2UU.AMReXSymb2Expr())
fileRHS.write(AbarUU.AMReXSymb2Expr())

486

In [15]:
alphaFunc = stVar('alpha')
alphaFunc.expr = 1/(M1.symb/(2*rc1.symb)+M2.symb/(2*rc2.symb))

In [16]:
fileRHS.write(alphaFunc.AMReXSymb2Expr())

72

In [17]:
beta = stVar('beta')

for i in range(3):
    for j in range(3):
        beta.expr += 1/8*(alphaFunc.symb**7)*AbarUU.symb[i][j]*AbarUU.symb[i][j]

In [18]:
fileRHS.write(beta.AMReXSymb2Expr())

468

In [19]:
RHS_u = stVar('u')

RHS_u.expr = 0
for i in range(3):
    RHS_u.expr += Dsymb(u.symb,str(i)+str(i))
    
RHS_u.expr += beta.symb*(alphaFunc.symb*(1+u.symb)+1)**(-7)

RHS_u.expr += Dsymb(u.symb,'KO')

In [20]:
fileRHS.write(RHS_u.AMReXSetRHS())

103

In [21]:
fileRHS.write(stfw.Closer())
fileRHS.close()

# Diagnostics

In [22]:
fileDIAG = open(path+"ET_Integration_Diag_K.H", "w+")

In [23]:
fileDIAG.write(stfw.Diag_Header())

754

In [24]:
fileDIAG.write(u.AMReXSymb2State())

53

In [25]:
fileDIAG.write(r.AMReXSymb2Expr())
fileDIAG.write(c1.AMReXSymb2Expr())
fileDIAG.write(c2.AMReXSymb2Expr())
fileDIAG.write(M1.AMReXSymb2Expr())
fileDIAG.write(M2.AMReXSymb2Expr())
fileDIAG.write(P1.AMReXSymb2Expr())
fileDIAG.write(P2.AMReXSymb2Expr())
fileDIAG.write(S1.AMReXSymb2Expr())
fileDIAG.write(S2.AMReXSymb2Expr())
fileDIAG.write(xc1.AMReXSymb2Expr())
fileDIAG.write(xc2.AMReXSymb2Expr())
fileDIAG.write(rc1.AMReXSymb2Expr())
fileDIAG.write(rc2.AMReXSymb2Expr())
fileDIAG.write(n1.AMReXSymb2Expr())
fileDIAG.write(n2.AMReXSymb2Expr())

108

In [26]:
fileDIAG.write(Abar1UU.AMReXSymb2Expr())
fileDIAG.write(Abar2UU.AMReXSymb2Expr())
fileDIAG.write(AbarUU.AMReXSymb2Expr())

486

In [27]:
fileDIAG.write(alphaFunc.AMReXSymb2Expr())

72

In [28]:
psi = stVar('psi')
psi.expr = 1+alphaFunc.symb**(-1)+u.symb

In [29]:
fileDIAG.write(psi.AMReXSymb2Expr())

46

In [30]:
stVar.declDiag = []  #We want a list of declared variables that we will cycle through when we convert our RHS equations.
chiDIAG = stVar('chi',diag = True) #Scalar variables like chi can be declared with stVar.
alphaDIAG = stVar('alpha',diag = True)

In [31]:
fileDIAGVARS = open(path+"ET_Integration_Diagnostic_Variables.H","w+")
fileDIAGVARS.write(stfw.VarString(stVar.declDiag))
fileDIAGVARS.close()

In [32]:
chiDIAG.expr = psi.symb**(-4)

In [33]:
fileDIAG.write(chiDIAG.AMReXSetDiag())

55

In [34]:
alphaDIAG.expr = psi.symb**(-2)

In [35]:
fileDIAG.write(alphaDIAG.AMReXSetDiag())

57

In [36]:
fileDIAG.write(stfw.Closer())
fileDIAG.close()

# Setup file

In [37]:
fileSETUP = open(path+"ET_Integration_Setup_K.H", "w+")
fileSETUP.write(stfw.Write_Setup_File(stVar.declState, stVar.declInit, stVar.declDiag, nghostcells))
fileSETUP.close()

# Post updating

In [38]:
filePOST = open(path+"ET_Integration_Post_Update_K.H", "w+")

In [39]:
filePOST.write(stfw.PostUpdate_Header())

708

In [40]:
filePOST.write(stfw.Closer())
filePOST.close()

# Init file

In [41]:
fileINIT = open(path+"ET_Integration_Init_K.H", "w+")

In [42]:
fileINIT.write(stfw.InitFromScratch_Header())

612

In [43]:
uInit = stVar('u')
uInit.expr = 0

In [44]:
fileINIT.write(uInit.AMReXSetState())

41

In [45]:
fileINIT.write(stfw.Closer())
fileINIT.close()

# AMR tagging

In [46]:
fileAMRTAG = open(path+"ET_Integration_AMR_Cell_Tag_K.H", "w+")

In [47]:
fileAMRTAG.write(stfw.AMRtagging_Header())

744

In [48]:
fileAMRTAG.write(u.AMReXSymb2State())

53

In [49]:
fileAMRTAG.write(r.AMReXSymb2Expr())
fileAMRTAG.write(c1.AMReXSymb2Expr())
fileAMRTAG.write(c2.AMReXSymb2Expr())
fileAMRTAG.write(M1.AMReXSymb2Expr())
fileAMRTAG.write(M2.AMReXSymb2Expr())
fileAMRTAG.write(P1.AMReXSymb2Expr())
fileAMRTAG.write(P2.AMReXSymb2Expr())
fileAMRTAG.write(S1.AMReXSymb2Expr())
fileAMRTAG.write(S2.AMReXSymb2Expr())
fileAMRTAG.write(xc1.AMReXSymb2Expr())
fileAMRTAG.write(xc2.AMReXSymb2Expr())
fileAMRTAG.write(rc1.AMReXSymb2Expr())
fileAMRTAG.write(rc2.AMReXSymb2Expr())
fileAMRTAG.write(n1.AMReXSymb2Expr())
fileAMRTAG.write(n2.AMReXSymb2Expr())

108

In [50]:
fileAMRTAG.write(alphaFunc.AMReXSymb2Expr())
fileAMRTAG.write(psi.AMReXSymb2Expr())

46

In [51]:
alphaAMRTAG = stVar('alphaTAG')

In [52]:
alphaAMRTAG.expr = psi.symb**(-2)

In [53]:
fileAMRTAG.write(alphaAMRTAG.AMReXSymb2Expr())

51

In [54]:
fileAMRTAG.write(TagCondition(alphaAMRTAG, '<','symb'))

43

In [55]:
fileAMRTAG.write(stfw.Closer())
fileAMRTAG.close()

# Gridding, Parameters, and Inputs

In [56]:
import STvARInputs as sti

In [57]:
pathInputs = "../Exec/InitialDataSolver/"
INPUTS = open(pathInputs + "inputs", "w+")

In [58]:
#STOPPING CRITERIA
MAX_STEP = 200
MAX_TIME = 100.0

#Equation type (default hyperbolic) and Courant Factor.
CFL = 0.005

#Geometry
NUM_COARSE_CELLS = [64, 64, 64]
LOWER_BOUNDS = [-40.0, -40.0, -40.0]
UPPER_BOUNDS = [40.0, 40.0, 40.0]
PERIODICITY = [0, 0, 0]

#AMR Refinement Parameters
NUM_LEVELS = 6
MAX_NUM_GRID_CELLS = 32
REF_RATIO = 2
NUM_BUFFER_CELLS = 1
BLOCKING_FACTOR = 8
REGRID_PERIOD = 2

#Coarse/Fine boundary interpolation procedure
INTERPOLATION_TYPE = 6

#TAGGING_THRESHOLDS = [0.003, 0.005, 0.007, 0.009, 0.011]
TAGGING_THRESHOLDS = [0.7, 0.6, 0.5, 0.4, 0.3]
ERROR_COMP = 0

#Problem specific Boundary Conditions
BC_UPPER = [2, 2, 2]
BC_LOWER = [2, 2, 2]

#How often to plot states, plot diagnostics, and write checkpoint files
PLOT_INT = 1
DIAG_INT = 10
CHECKPOINT_INT = 10

#Restart file names
RESTART_FILE = "initial_data_chk"
IS_INITIAL_DATA = 1

#Integration type.
RUNGE_KUTTA_TYPE = 4

In [59]:
INPUTS.write(sti.StoppingCriteria(MAX_STEP, MAX_TIME))
INPUTS.write(sti.Geometry(NUM_COARSE_CELLS, LOWER_BOUNDS, UPPER_BOUNDS, PERIODICITY))
INPUTS.write(sti.AMRVerbosity())
INPUTS.write(sti.RefinementPar(NUM_LEVELS, MAX_NUM_GRID_CELLS, REF_RATIO, NUM_BUFFER_CELLS, BLOCKING_FACTOR, REGRID_PERIOD))
INPUTS.write(sti.Interpolation(INTERPOLATION_TYPE))
INPUTS.write(sti.BoundaryConditions(BC_LOWER, BC_UPPER))
INPUTS.write(sti.TaggingThresholds(TAGGING_THRESHOLDS, ERROR_COMP))
INPUTS.write(sti.ProblemType(CFL))
INPUTS.write(sti.PlottingCriteria(PLOT_INT, DIAG_INT, CHECKPOINT_INT))
INPUTS.write(sti.IntegrationType(RUNGE_KUTTA_TYPE))

711

In [60]:
INPUTS.close()